<center>
  <h1>Skalu - Line and Rectangle Detection</h1>
</center>

This notebook detects horizontal lines and rectangles in images and PDF files using computer vision.

## Instructions

1.  Run the **Setup Environment** cell first.
2.  Upload your files (images or PDFs) using the file browser on the left sidebar.
3.  In the **Process Files** cell, enter the path to your input file or folder.
4.  Adjust the detection parameters as needed.
5.  Run the **Process Files** cell to start the analysis.
6.  Results, including a JSON file and annotated images, will be saved in the `output` folder and download links will appear.

In [ ]:
#@title ## 1. Setup Environment
#@markdown This cell will install the necessary libraries for processing. Please run it once.
import os
import sys
import subprocess

# Install or update required packages from requirements.txt
!pip install -q opencv-python numpy tqdm PyMuPDF Pillow ipywidgets

# Create directory for output
!mkdir -p output

print("✅ Environment setup complete!")

In [ ]:
# Skalu - Detection Tool Code
import cv2
import os
import sys
import json
import fitz  # PyMuPDF
import numpy as np
from tqdm.notebook import tqdm
from PIL import Image as PILImage
from IPython.display import display, HTML, Image
import base64

def detect_horizontal_lines(image, min_line_width_ratio=0.2, max_line_height=10, debug_dir=None):
    h, w = image.shape[:2]
    if debug_dir: os.makedirs(debug_dir, exist_ok=True)

    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) if len(image.shape) == 3 else image.copy()
    if debug_dir: cv2.imwrite(os.path.join(debug_dir, "step_01_gray.png"), gray)

    _, bw_otsu = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)
    if debug_dir: cv2.imwrite(os.path.join(debug_dir, "step_02_otsu.png"), bw_otsu)

    bw_adapt = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 15, 8)
    if debug_dir: cv2.imwrite(os.path.join(debug_dir, "step_03_adaptive.png"), bw_adapt)

    bw = cv2.bitwise_or(bw_otsu, bw_adapt)
    if debug_dir: cv2.imwrite(os.path.join(debug_dir, "step_04_union.png"), bw)

    orig_min_width = int(min_line_width_ratio * w)
    open_width = max(int(orig_min_width * 0.8), 1)
    horiz_kern = cv2.getStructuringElement(cv2.MORPH_RECT, (open_width, 1))
    opened = cv2.morphologyEx(bw, cv2.MORPH_OPEN, horiz_kern, iterations=1)

    contours, _ = cv2.findContours(opened, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    lines = []
    for c in contours:
        x, y, cw, ch = cv2.boundingRect(c)
        if cw >= orig_min_width and ch <= max_line_height:
            lines.append({"x":x, "y":y, "width":cw, "height":ch})

    lines.sort(key=lambda L: L["y"])
    return lines

def detect_rectangles(image, min_rect_area_ratio=0.001, max_rect_area_ratio=0.5, debug_dir=None):
    h, w = image.shape[:2]
    img_area = h * w
    min_area = int(min_rect_area_ratio * img_area)
    max_area = int(max_rect_area_ratio * img_area)

    if debug_dir: os.makedirs(debug_dir, exist_ok=True)

    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) if len(image.shape) == 3 else image.copy()
    if debug_dir: cv2.imwrite(os.path.join(debug_dir, "rect_01_gray.png"), gray)

    edges = cv2.Canny(gray, 50, 150)
    if debug_dir: cv2.imwrite(os.path.join(debug_dir, "rect_02_edges.png"), edges)

    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
    dilated = cv2.dilate(edges, kernel, iterations=1)
    if debug_dir: cv2.imwrite(os.path.join(debug_dir, "rect_03_dilated.png"), dilated)

    contours, _ = cv2.findContours(dilated, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    rectangles = []
    for c in contours:
        area = cv2.contourArea(c)
        if not (min_area < area < max_area):
            continue
        
        peri = cv2.arcLength(c, True)
        approx = cv2.approxPolyDP(c, 0.02 * peri, True)
        
        if len(approx) == 4:
            x, y, w, h = cv2.boundingRect(approx)
            if w > 5 and h > 5:
                rectangles.append({"x": x, "y": y, "width": w, "height": h})
    
    rectangles.sort(key=lambda r: r["width"] * r["height"], reverse=True)
    return rectangles

def get_image_dpi(image_path):
    try:
        with PILImage.open(image_path) as img:
            dpi = img.info.get('dpi', (0, 0))
            return int(dpi[0]), int(dpi[1])
    except Exception as e:
        print(f"Warning: Could not read DPI for {image_path}: {e}")
        return 0, 0

def draw_detections(image, horizontal_lines=None, rectangles=None):
    debug = image.copy()
    if horizontal_lines:
        for i, L in enumerate(horizontal_lines):
            x, y, w, h = L["x"], L["y"], L["width"], L["height"]
            cv2.rectangle(debug, (x, y), (x + w, y + h), (0, 255, 0), 2)
            cv2.putText(debug, f"Line #{i+1}", (x, y - 6), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1)
    if rectangles:
        for i, R in enumerate(rectangles):
            x, y, w, h = R["x"], R["y"], R["width"], R["height"]
            cv2.rectangle(debug, (x, y), (x + w, y + h), (255, 0, 0), 2)
            cv2.putText(debug, f"Rect #{i+1}", (x, y - 6), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 1)
    return debug

def create_download_button(file_path, button_text=None):
    if button_text is None: button_text = f"Download {os.path.basename(file_path)}"
    with open(file_path, 'rb') as f: data = f.read()
    b64 = base64.b64encode(data).decode()
    button_html = f'''
    <a href="data:application/octet-stream;base64,{b64}" download="{os.path.basename(file_path)}">
        <button style="font-size: 14px; padding: 5px 15px; background-color: #4CAF50; color: white; 
                 border: none; border-radius: 4px; cursor: pointer;">{button_text}</button>
    </a>'''
    return HTML(button_html)

print("✅ Detection functions are defined.")

In [ ]:
#@title ## 2. Process Files
#@markdown ### Input and Output Configuration
input_path = "" #@param {type:"string"}
output_json_path = "" #@param {type:"string"}
#@markdown --- 
#@markdown ### Detection Parameters
min_line_width_ratio = 0.2 #@param {type:"slider", min:0.1, max:0.8, step:0.05}
max_line_height = 10 #@param {type:"slider", min:1, max:50, step:1}
min_rect_area_ratio = 0.001 #@param {type:"slider", min:0.0001, max:0.1, step:0.0001, format:"%.4f"}
max_rect_area_ratio = 0.5 #@param {type:"slider", min:0.1, max:1.0, step:0.05}
#@markdown --- 
#@markdown ### Options
save_visualization = True #@param {type:"boolean"}
debug_dir = "" #@param {type:"string"}
#@markdown _(Optional) Set a directory name (e.g., 'debug_steps') to save intermediate processing images._

def process_single_image(image_path, out_json, params, viz, dbg_dir):
    img = cv2.imread(image_path)
    if img is None: 
        print(f"Warning: Cannot read {image_path}, skipping.")
        return

    lines = detect_horizontal_lines(img, params['min_line_width_ratio'], params['max_line_height'], dbg_dir)
    rects = detect_rectangles(img, params['min_rect_area_ratio'], params['max_rect_area_ratio'], dbg_dir)
    dpi_x, dpi_y = get_image_dpi(image_path)
    
    result = {"result": {os.path.basename(image_path): {"dpi": {"x": dpi_x,"y": dpi_y,"height": img.shape[0],"width":  img.shape[1]}}},
              "detection_params": params}
    if lines: result["result"][os.path.basename(image_path)]["horizontal_lines"] = lines
    if rects: result["result"][os.path.basename(image_path)]["rectangles"] = rects

    with open(out_json, "w", encoding="utf-8") as f: json.dump(result, f, indent=4)
    print(f"Saved detection for {image_path} → {out_json}")

    if viz:
        debug_img = draw_detections(img, lines, rects)
        viz_path = os.path.join(os.path.dirname(out_json), os.path.splitext(os.path.basename(out_json))[0] + "_detected.jpg")
        cv2.imwrite(viz_path, debug_img)
        print(f"Saved visualization → {viz_path}")
        display(Image(viz_path, width=600))

def process_pdf(pdf_path, out_json, params, viz, dbg_dir):
    doc = fitz.open(pdf_path)
    pages_data = []
    viz_paths = []
    for page_num in tqdm(range(len(doc)), desc="Processing PDF pages"):
        page = doc.load_page(page_num)
        pix = page.get_pixmap(matrix=fitz.Matrix(200/72, 200/72))
        img = cv2.imdecode(np.frombuffer(pix.tobytes("png"), np.uint8), cv2.IMREAD_COLOR)
        if img is None: continue
        
        page_dbg_dir = os.path.join(dbg_dir, f"page_{page_num + 1}") if dbg_dir else None
        lines = detect_horizontal_lines(img, params['min_line_width_ratio'], params['max_line_height'], page_dbg_dir)
        rects = detect_rectangles(img, params['min_rect_area_ratio'], params['max_rect_area_ratio'], page_dbg_dir)
        
        if lines or rects:
            page_result = {"page": page_num + 1, "width": img.shape[1], "height": img.shape[0]}
            if lines: page_result["horizontal_lines"] = lines
            if rects: page_result["rectangles"] = rects
            pages_data.append(page_result)

        if viz:
            debug_img = draw_detections(img, lines, rects)
            viz_path = os.path.join(os.path.dirname(out_json), f"{os.path.splitext(os.path.basename(pdf_path))[0]}_page_{page_num + 1}_detected.jpg")
            cv2.imwrite(viz_path, debug_img)
            viz_paths.append(viz_path)
    doc.close()

    result = {"dpi": {"x": 200, "y": 200}, "pages": pages_data, "detection_params": params}
    with open(out_json, "w", encoding="utf-8") as f: json.dump(result, f, indent=4)
    print(f"Saved PDF detection results → {out_json}")
    if viz:
        print("Displaying first 5 visualizations:")
        for p in viz_paths[:5]:
           display(HTML(f"<h4>{os.path.basename(p)}</h4>"))
           display(Image(p, width=600))

def process_folder(folder_path, out_json, params, viz, dbg_dir):
    result = {"result": {}}
    exts = [".jpg", ".jpeg", ".png", ".bmp", ".tiff", ".webp"]
    imgs = sorted([f for f in os.listdir(folder_path) if any(f.lower().endswith(e) for e in exts)])
    if not imgs: 
        print(f"No supported images found in {folder_path}")
        return

    viz_paths = []
    for fn in tqdm(imgs, desc="Processing folder"):
        path = os.path.join(folder_path, fn)
        img = cv2.imread(path)
        if img is None: continue

        img_dbg_dir = os.path.join(dbg_dir, os.path.splitext(fn)[0]) if dbg_dir else None
        lines = detect_horizontal_lines(img, params['min_line_width_ratio'], params['max_line_height'], img_dbg_dir)
        rects = detect_rectangles(img, params['min_rect_area_ratio'], params['max_rect_area_ratio'], img_dbg_dir)
        
        dpi_x, dpi_y = get_image_dpi(path)
        result["result"][fn] = {"dpi": {"width": img.shape[1], "height": img.shape[0]}}
        if dpi_x != 0: result["result"][fn]["dpi"]["x"] = dpi_x
        if dpi_y != 0: result["result"][fn]["dpi"]["y"] = dpi_y
        if lines: result["result"][fn]["horizontal_lines"] = lines
        if rects: result["result"][fn]["rectangles"] = rects

        if viz:
            debug_img = draw_detections(img, lines, rects)
            viz_path = os.path.join(os.path.dirname(out_json), f"{os.path.splitext(fn)[0]}_detected.jpg")
            cv2.imwrite(viz_path, debug_img)
            viz_paths.append(viz_path)
            
    result["detection_params"] = params
    with open(out_json, "w", encoding="utf-8") as f: json.dump(result, f, indent=4)
    print(f"Saved all results → {out_json}")
    if viz:
        print("Displaying first 5 visualizations:")
        for p in viz_paths[:5]:
           display(HTML(f"<h4>{os.path.basename(p)}</h4>"))
           display(Image(p, width=600))

# Main execution logic
if not input_path:
    print("❌ Please provide an input path for a file or folder.")
else:
    params = {
        'min_line_width_ratio': min_line_width_ratio,
        'max_line_height': max_line_height,
        'min_rect_area_ratio': min_rect_area,
        'max_rect_area_ratio': max_rect_area
    }

    # Auto-choose output JSON path if not specified
    out = output_json_path
    output_dir = 'output'
    if not out:
        base_name = os.path.splitext(os.path.basename(input_path))[0] if os.path.isfile(input_path) else os.path.basename(input_path)
        out = os.path.join(output_dir, f"{base_name}_structures.json")
    
    os.makedirs(os.path.dirname(out), exist_ok=True)
    final_debug_dir = os.path.join(output_dir, debug_dir) if debug_dir else None

    if os.path.isfile(input_path):
        if input_path.lower().endswith('.pdf'):
            process_pdf(input_path, out, params, save_visualization, final_debug_dir)
        else:
            process_single_image(input_path, out, params, save_visualization, final_debug_dir)
    elif os.path.isdir(input_path):
        process_folder(input_path, out, params, save_visualization, final_debug_dir)
    else:
        print(f"Error: Input path {input_path} is not a valid file or directory.")
        
    # Display download buttons for the results
    if os.path.exists(out):
        display(HTML("<h3>Download Results</h3>"))
        display(create_download_button(out, "Download JSON Results"))
        # Create a zip file of all generated files in the output directory for easy download
        zip_path = os.path.join(output_dir, 'skalu_results.zip')
        # The -j flag in zip command stores files without directory structure inside the zip
        os.system(f'zip -q -j {zip_path} {output_dir}/*')
        if os.path.exists(zip_path):
            display(create_download_button(zip_path, "Download All Results (ZIP)"))

## How to Use the Results

The output `_structures.json` file contains detailed information about each detected line and rectangle. The structure of the JSON will vary slightly depending on whether you processed a single image, a folder, or a PDF.

### Example for a Single Image or Folder:

```json
{
    "result": {
        "my_image.png": {
            "dpi": {
                "x": 300,
                "y": 300,
                "height": 2200,
                "width": 1700
            },
            "horizontal_lines": [
                {"x": 150, "y": 400, "width": 1400, "height": 3},
                ...
            ],
            "rectangles": [
                {"x": 200, "y": 500, "width": 500, "height": 150},
                ...
            ]
        }
    },
    "detection_params": { ... }
}
```

### Example for a PDF:
```json
{
    "dpi": {"x": 200, "y": 200},
    "pages": [
        {
            "page": 1,
            "width": 1654,
            "height": 2339,
            "horizontal_lines": [
                {"x": 120, "y": 350, "width": 1000, "height": 2}
            ],
            "rectangles": []
        },
        {
            "page": 2,
            ...
        }
    ],
    "detection_params": { ... }
}
```

You can use this structured data for:
- Document layout analysis
- Form field and table extraction
- Automated data entry pre-processing
- Quality control in document digitization